# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.110270e+02     1.522802e+00
 * time: 0.08912205696105957
     1     1.119121e+01     8.670926e-01
 * time: 0.8706469535827637
     2    -1.146492e+01     9.233957e-01
 * time: 1.0244231224060059
     3    -3.384943e+01     7.516409e-01
 * time: 1.183448076248169
     4    -4.733740e+01     5.488358e-01
 * time: 1.3272840976715088
     5    -5.692643e+01     2.114832e-01
 * time: 1.4670960903167725
     6    -5.991794e+01     1.375016e-01
 * time: 1.5707809925079346
     7    -6.101682e+01     5.046242e-02
 * time: 1.6814188957214355
     8    -6.140238e+01     6.611295e-02
 * time: 1.7917261123657227
     9    -6.165282e+01     5.052839e-02
 * time: 1.8968100547790527
    10    -6.184438e+01     2.570875e-02
 * time: 2.007923126220703
    11    -6.199556e+01     2.271569e-02
 * time: 2.1119019985198975
    12    -6.205983e+01     1.670081e-02
 * time: 2.222745895385742
    13    -6.212431e+01     1.449050e-02
 * time: 2.331748008728

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557625
    AtomicNonlocal      14.8522605
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485356 
    Xc                  -19.3336814

    total               -62.261666459034
